In [1]:
import time
import pandas as pd
import traceback
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# ▼ 入出力ファイル
input_file = "unique_titles.csv"
output_file = "apps_release_dates.csv"

# ▼ Chrome起動設定
options = Options()
options.add_argument("--start-maximized")
# options.add_argument("--headless")  # 必要なら有効化

driver = webdriver.Chrome(options=options)

# ▼ タイトル読み込み（30件だけに限定）
df = pd.read_csv(input_file).head(30).copy()
df["発売日"] = None
total = len(df)

# ▼ メイン処理ループ
for idx, row in df.iterrows():
    title = row["タイトル"]
    print(f"[{idx+1}/{total}] 検索中：{title}")

    try:
        # トップページに遷移 → 検索実行
        driver.get("https://www.4gamer.net/")
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, "word")))

        search_box = driver.find_element(By.NAME, "word")
        search_box.clear()
        search_box.send_keys(title)
        search_box.send_keys(Keys.RETURN)

        # 検索結果の表示を待つ（失敗したらスキップ）
        try:
            WebDriverWait(driver, 10).until(
                EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2 > a[href^="/games/"]'))
            )
            links = driver.find_elements(By.CSS_SELECTOR, 'h2 > a[href^="/games/"]')
        except TimeoutException:
            print(" → 検索結果が見つかりません（タイムアウト）")
            continue

        game_links = [link for link in links if "/games/" in (link.get_attribute("href") or "")]
        if not game_links:
            print(" → ゲーム紹介ページが見つかりません")
            continue

        # ゲーム紹介ページへ遷移 → 発売日を抽出
        release_date = None
        for link in game_links:
            try:
                url = link.get_attribute("href")
                if not url.startswith("http"):
                    url = "https://www.4gamer.net" + url

                driver.get(url)
                WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.TAG_NAME, "th"))
                )

                th_list = driver.find_elements(By.TAG_NAME, "th")
                for th in th_list:
                    if "発売日" in th.text:
                        try:
                            td = th.find_element(By.XPATH, "./following-sibling::td[2]")
                            release_date = td.text.strip()
                            break
                        except:
                            continue

                if release_date:
                    df.at[idx, "発売日"] = release_date
                    print(f" → 発売日：{release_date}")
                    break

            except Exception as sub_e:
                print(" → ページ処理中エラー:")
                traceback.print_exc()
                continue

        if not release_date:
            print(" → 発売日が見つかりません")

    except Exception as e:
        print(" → メイン処理エラー:")
        traceback.print_exc()
        continue

# ▼ 終了処理
driver.quit()
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"\n✅ 処理完了！→ {output_file} に保存されました。")


[1/30] 検索中：スーパーガンダムロワイヤル
 → 発売日：2015/10/16
[2/30] 検索中：武器よさらば
 → 発売日：2017/03/30
[3/30] 検索中：スーパーロボット大戦X-Ω
 → 発売日：2015/10/05
[4/30] 検索中：FINAL FANTASY GRANDMASTERS
 → 発売日：2015/10/01
[5/30] 検索中：ドラゴンプロジェクト
 → 発売日：2016/06/03
[6/30] 検索中：戦の海賊
 → 発売日：2015/08/27
[7/30] 検索中：誰ガ為のアルケミスト
 → 発売日：2016/01/28
[8/30] 検索中：デッキヒーローズ-本格派戦略カードゲーム-
 → 検索結果が見つかりません（タイムアウト）
[9/30] 検索中：美少女戦士セーラームーン　セーラームーンドロップス
 → 発売日：2015/09/03
[10/30] 検索中：モンスターハンター エクスプロア
 → 発売日：2015/09/29
[11/30] 検索中：夢色キャスト
 → 発売日：2024/09/29
[12/30] 検索中：グリムノーツ
 → 発売日：2016/01/21
[13/30] 検索中：DAME×PRINCE -ダメ王子たちとのドタバタ恋愛ADV
 → 発売日：2016/03/31
[14/30] 検索中：かんぱにガールズ ファンタジーRPG
 → 検索結果が見つかりません（タイムアウト）
[15/30] 検索中：デジモンリンクス
 → 発売日：2016/03/24
[16/30] 検索中：BFBチャンピオンズ2.0
 → 発売日：2016/06/29
[17/30] 検索中：フェアリーテイル 極・魔法乱舞
 → 発売日：2015/12/04
[18/30] 検索中：12オーディンズ - 王道RPG
 → 検索結果が見つかりません（タイムアウト）
[19/30] 検索中：ラストピリオド - 終わりなき螺旋の物語 -
 → 発売日：2016/05/10
[20/30] 検索中：聖闘士星矢 ゾディアック ブレイブ
 → 発売日：2016/01/28
[21/30] 検索中：ブレイブファンタジア【まったり＆簡単操作の爽快RPG】
 → 検索結果が見つかりません（タイムアウト）
[22/30] 検索中：